In [1]:
%pip install dbfread

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dbfread import DBF
import psycopg2
import csv
from io import StringIO
from datetime import datetime
from dbfread import DBF

In [3]:
import psycopg2
import psycopg2.extras

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    dbname="bytsscom_unmsm",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
conn.autocommit = True
cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

In [4]:
from decimal import Decimal

In [6]:
def get_expediente_fase():
    cur.execute(
    """
    SELECT c.ano_eje,
       cert.id_area,
       cf.sys_id_certificacion,
       cs.fecha_creacion_clt,
       --cs.fecha_bd_oracle,
       c.certificado,
       cf.secuencia,
       f.siaf_codigo                        AS sys_id_cuente,
       f.abre_fuente,
        mt.id_meta_institucional,
        mt.sec_func,
        mp.nomb_met_ins,
       cla.id_clasificador                AS sys_ID_GENERICA,
       substring(cm.sys_cod_clasif, 1, 2) as GENERICA,
       CASE cf.secuencia_padre
           WHEN '0000' THEN 'DP'
           WHEN '0001' THEN 'CA'
           ELSE '' END                    AS sys_TIPO_CERT,
       cs.cod_doc,
       cs.num_doc,
       cf.glosa,
       --sum(cs.monto) AS MONTO_INICIAL,
       --sum(cs.monto_comprometido) AS MONTO_COMPROMETIDO,
       sum(cs.monto_nacional)             AS MONTO_NACIONAL,
       cm.sys_id_clasificador             AS sys_ID_CLASIFICADOR,
       cm.sys_cod_clasif                  as CLASIFICADOR,
       sum(cm.monto_nacional)             AS MONTO_CLASIFICADOR,
       cs.estado_envio,
       cs.estado_registro
FROM bytsscom_bytsiaf.certificado c
         INNER JOIN bytsscom_bytsiaf.certificado_fase cf
                    ON c.certificado = cf.certificado
         INNER JOIN bytsscom_bytcore.fuente f
                    ON cf.fuente_financ = f.siaf_codigo
                        AND cf.ano_eje = '2025'
         INNER JOIN bytsscom_bytsiaf.certificado_secuencia cs
                    ON c.certificado = cs.certificado
                        AND cf.secuencia = cs.secuencia
                        AND cs.estado_registro = 'A'
                        AND cs.ano_eje = '2025'
         INNER JOIN bytsscom_bytsiaf.certificado_meta cm
                    ON cf.certificado = cm.certificado
                        AND cf.secuencia = cm.secuencia
                        AND cm.ano_eje = '2025'
         LEFT JOIN bytsscom_bytcore.meta_institucional_siaf mt
                    ON cm.sec_func = mt.sec_func
                    AND mt.id_anio = '2025'::int
         INNER JOIN bytsscom_bytcore.meta_institucional mp
                    ON mt.id_meta_institucional = mp.id_meta_institucional
         INNER JOIN bytsscom_bytcore.clasificador cla
                    ON substring(cm.sys_cod_clasif, 1, 2) = cla.cod_clasif
         LEFT JOIN (SELECT distinct c.id_certificacion,
                                    m.id_area
                    FROM bytsscom_bytsig.vw_certificacion c
                             inner join bytsscom_bytsig.memo_requerimiento m
                                        on c.id_memo_requerimiento = m.id_memo_requerimiento
                    where c.id_anio = 2025
                      AND c.esta_cert = 'A') cert
                   ON cf.sys_id_certificacion = cert.id_certificacion
WHERE c.ano_eje = '2025' and cf.secuencia_padre = '0000'
--AND cf.certificado = '0000006928'
GROUP BY c.ano_eje,
         cert.id_area,
         cf.sys_id_certificacion,
         cs.fecha_creacion_clt,
         --cs.fecha_bd_oracle,
         c.certificado,
         cf.secuencia,
         f.id_fuente,
         f.abre_fuente,
        mt.id_meta_institucional,
        mt.sec_func,
        mp.nomb_met_ins,
         cla.id_clasificador,
         cm.sys_id_clasificador,
         cm.sys_cod_clasif,
         --cs.correlativo,
         cf.secuencia_padre,
         cs.cod_doc,
         cs.num_doc,
         cf.glosa,
         cs.estado_envio,
         cs.estado_registro
ORDER BY c.certificado,
         cf.secuencia,
         cm.sys_cod_clasif;
    """,)
    return cur.fetchall()
  

In [7]:
resultados = get_expediente_fase()
print(resultados)
for fila in resultados:
    print(fila)

[['2025', None, 444186, datetime.date(2025, 1, 8), '0000000001', '0001', '00', '00-RO', 17, '0034', 'EJERCICIO DE LA DOCENCIA UNIVERSITARIA /  / ', 856, '21', 'DP', '086', '00005-2024-OGRRHH   ', 'PLANILLA CAS RO', Decimal('8165489.00'), 1894, '21 113 1 1', Decimal('966793.00'), 'T', 'A'], ['2025', None, 444186, datetime.date(2025, 1, 8), '0000000001', '0001', '00', '00-RO', 31, '0050', 'ADMINISTRACION DE LOS RECURSOS HUMANOS, FINANCIEROS Y LOGISTICOS /  / ', 856, '21', 'DP', '086', '00005-2024-OGRRHH   ', 'PLANILLA CAS RO', Decimal('8165489.00'), 1894, '21 113 1 1', Decimal('3666579.00'), 'T', 'A'], ['2025', None, 444186, datetime.date(2025, 1, 8), '0000000001', '0001', '00', '00-RO', 139, '0051', ' /  / ', 856, '21', 'DP', '086', '00005-2024-OGRRHH   ', 'PLANILLA CAS RO', Decimal('8165489.00'), 1894, '21 113 1 1', Decimal('1507200.00'), 'T', 'A'], ['2025', None, 444186, datetime.date(2025, 1, 8), '0000000001', '0001', '00', '00-RO', 31, '0050', 'ADMINISTRACION DE LOS RECURSOS HUMANOS